In [ ]:
# Autoload when refreshing notebook
%load_ext autoreload
%autoreload 2

import numpy as np
import h5py
from scipy.io import loadmat
import pandas as pd
import re
import matplotlib.pyplot as plt
from types import SimpleNamespace
import scipy
import warnings
from scipy.ndimage import median_filter, gaussian_filter
from scipy.optimize import curve_fit

# import Python functions 
import sys
sys.path.append('../../')

from Python_Functions.functions import matstruct_to_dict, extractDAQBSAScalars, apply_tcav_zeroing_filter
from Python_Functions.gmm import biGaussian_image_from_flattened_params, flatten_biGaussian_params, unflatten_biGaussian_params

### MLP 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
from scipy.io import loadmat 
import re
import os 
import joblib
# Assumed: commonIndexFromSteps, extractDAQBSAScalars, and other helper functions are available

# ----------------------------------------------------------------------
# 0. Load the model with joblib
# Ex: MLP_LPS_GMM_E338_12710_20251031_194740


joblib_file = '../../model/LPS/MLP_LPS_GMM_Forest_E338_12710_20251103_082505.pkl'  # Modify as needed
model = joblib.load(joblib_file)
iz_scaler = pickle.load(open('../../model/LPS/E338_12710_iz_scaler_GMM_20251103_082505.pkl', 'rb'))
# ----------------------------------------------------------------------
# 1. Define the list of (experiment, runname, step_identifier) pairs to test the model on.
# ----------------------------------------------------------------------
run_pairs = [
    ('E338', '12716', 1),  # Example pairs, modify this list
    #('E300', '12431', 1),
    # Add more pairs here...
]

# ----------------------------------------------------------------------
# 2. Initialize lists for concatenation
# ----------------------------------------------------------------------
all_predictors = []
all_indices = []

print("Starting multi-run data loading and concatenation...")

# ----------------------------------------------------------------------
# 3. Loop through runs, load data, and concatenate
# ----------------------------------------------------------------------
for experiment, runname, step_id in run_pairs:
        
    # --- B. Load and Filter Predictor Data (BSA Scalars) ---
    
    # 1. Load data_struct
    dataloc = f'../../data/raw/{experiment}/{experiment}_{runname}/{experiment}_{runname}.mat'
    try:
        mat = loadmat(dataloc,struct_as_record=False, squeeze_me=True)
        data_struct = mat['data_struct']
    except FileNotFoundError:
        print(f"Skipping {experiment}_{runname}: .mat file not found at {dataloc}")
        continue

    # 2. Extract full BSA scalars (filtered by step_list if needed)
    # Don't filter by common index here, we'll do it with the goodShots scalar common index loaded from the file
    bsaScalarData, bsaVars = extractDAQBSAScalars(data_struct, filter_index=False)
    bsaScalarData = apply_tcav_zeroing_filter(bsaScalarData, bsaVars)

    ampl_idx = next(i for i, var in enumerate(bsaVars) if 'TCAV_LI20_2400_A' in var)
    xtcavAmpl = bsaScalarData[ampl_idx, :]

    phase_idx = next(i for i, var in enumerate(bsaVars) if 'TCAV_LI20_2400_P' in var)
    xtcavPhase = bsaScalarData[phase_idx, :]
    xtcavOffShots = xtcavAmpl<0.1
    xtcavPhase[xtcavOffShots] = 0 #Set this for ease of plotting
    fig, ax1 = plt.subplots()
    ax1.plot(xtcavAmpl, label='Amplitude', color='b')
    ax1.set_ylabel('XTCAV Ampl [MV]', color='b')
    ax1.tick_params(axis='y', labelcolor='b')

    ax2 = ax1.twinx()
    ax2.plot(xtcavPhase, label='Phase', color='r')
    ax2.set_ylabel('XTCAV Phase [deg]', color='r')
    ax2.tick_params(axis='y', labelcolor='r')

    plt.title('XTCAV Amplitude and Phase')
    plt.show()
    # 5. Filter BSA data using the final index
    # goodShots_scal_common_index is 1 based indexing from MATLAB, convert to 0 based
    bsaScalarData_filtered = bsaScalarData
    
    # 6. Construct the predictor array
    predictor_current = np.vstack(bsaScalarData_filtered).T
    
    # C. Append to master lists
    all_predictors.append(predictor_current)
    
# ----------------------------------------------------------------------
# 4. Concatenate and finalize arrays
# ----------------------------------------------------------------------
# Combine all data arrays from the runs
predictor_tmp = np.concatenate(all_predictors, axis=0)

# Set image half dimensions (should match preprocessing)
yrange = 50
xrange = 150
NCOMP = 10  # Number of GMM parameters
print("\n--- Final Concatenated Data Shapes ---")
print(f"Total Predictors (predictor): {predictor_tmp.shape}")



### Exclude BSA Variables

In [ ]:
from Python_Functions.functions import exclude_bsa_vars
excluded_var_idx = exclude_bsa_vars(bsaVars)
predictor_tmp_cleaned = np.delete(predictor_tmp, excluded_var_idx, axis=1)
bsaVars_cleaned = [var for i, var in enumerate(bsaVars) if i not in excluded_var_idx]
print(f"Predictor shape after excluding variables: {predictor_tmp_cleaned.shape}")


### Calibration

In [ ]:
# Define XTCAV calibration
krf = 239.26
cal = 1167 # um/deg  http://physics-elog.slac.stanford.edu/facetelog/show.jsp?dir=/2025/11/13.03&pos=2025-$
streakFromGUI = cal*krf*180/np.pi*1e-6#um/um
xtcalibrationfactor = 6.35e-15

isChargePV = [bool(re.search(r'TORO_LI20_2452_TMIT', pv)) for pv in bsaVars]
if isChargePV:
    # Extract charge data
    pvidx = [i for i, val in enumerate(isChargePV) if val]
    charge = bsaScalarData[pvidx, :][0] * 1.6e-19  # in C 
    charge_filtered = charge

# Set flag for current profile fitting. If True, the current profile will be used to refine the GMM fit.
do_current_profile = True
NCOMP = 10

In [ ]:
    
# --- Original scaling and splitting logic follows ---

x_scaler = MinMaxScaler()
x_scaled = x_scaler.fit_transform(predictor_tmp_cleaned)

# all dataset is used for testing purpose here
x_test_scaled = x_scaled
ntest = np.arange(x_test_scaled.shape[0])

# Convert to PyTorch tensors
X_test = torch.tensor(x_test_scaled, dtype=torch.float32)
# Evaluate model
pred_test_scaled = model.predict(X_test)

# Inverse transform predictions
pred_test_full = iz_scaler.inverse_transform(pred_test_scaled)
#pca.inverse_transform(pred_test_scaled)
# Compute R² score
def r2_score(true, pred):
    RSS = np.sum((true - pred)**2)
    TSS = np.sum((true - np.mean(true))**2)
    return 1 - RSS / TSS if TSS != 0 else s0

print("Test R² is Unknown (Iz_test_true not available)")


### Plot prediction

In [ ]:
from ipywidgets import interact, IntSlider, Layout
def plot_xtcav_image_pred(idx):
    fig, (ax2, cx1) = plt.subplots(1,2,figsize=(8, 6), gridspec_kw={'width_ratios': [1, 0.02]})
    pred_params = pred_test_full.T.reshape(NCOMP,pred_test_full.shape[0])[:,idx]
    pred_im = biGaussian_image_from_flattened_params(pred_params, total_charge=charge_filtered[idx] if do_current_profile else None, yrange=yrange, xrange=xrange)
    pred_im = pred_im.T
    im2 = ax2.imshow(pred_im*1e15, cmap = "jet", extent=(-xrange*xtcalibrationfactor*1e15, xrange*xtcalibrationfactor*1e15, 0, 2*yrange), aspect='auto')
    ax2.set(xlabel = "Time [fs]")
    ax2.set(ylabel = "y [pix]")
    ax2.set(title = f"LPS Prediction: {experiment}_{runname}, Shot Number: {ntest[idx]}")
    cbar = fig.colorbar(im2, cax=cx1, fraction=0.16, pad=0.04)
    cbar.set_label("Current [A per y pixel]")
    plt.subplots_adjust(wspace=0.4)
    # plt.tight_layout()
    # fig.show()
    # Also plot R² value for this index
    print(f"Displaying prediction for index: {idx}")
    print(f"Parameters: {unflatten_biGaussian_params(pred_params)}")

# Create slider
interact(plot_xtcav_image_pred, idx=IntSlider(min=0, max=pred_test_full.shape[0]-1, step=1, value=0, layout=Layout(width='1000px')))

In [ ]:
from ipywidgets import interact, IntSlider, Layout
def plot_xtcav_image_pred_current(idx):
    FACTOR = 1e15
    fig, (ax0) = plt.subplots(1,1,figsize=(10, 6))

    x_grid = np.linspace(-xrange*xtcalibrationfactor*FACTOR, xrange*xtcalibrationfactor*FACTOR, xrange * 2)


    pred_params = pred_test_full.T.reshape(NCOMP,pred_test_full.shape[0])[:,idx]
    # Current profile is relative to zeta; transpose so that the drive bunch center agrees with the fit.
    pred_params_uf = unflatten_biGaussian_params(pred_params)
    pred_params = flatten_biGaussian_params(pred_params_uf)
    
    if isChargePV:
        # Normalize predicted image so that its integral matches the charge
        charge_value = charge_filtered[ntest[idx]]
        pred_im = biGaussian_image_from_flattened_params(pred_params, charge_value, xrange=xrange, yrange=yrange)
    else:
        pred_im = biGaussian_image_from_flattened_params(pred_params, xrange=xrange, yrange=yrange)
    pred_im = pred_im.T
    pred_im_proj = np.sum(pred_im, axis = 0) * FACTOR
    ax0.plot(x_grid, pred_im_proj, label = "prediction")
    # Fix x-axis limits
    ax0.set(xlim = (-xrange * xtcalibrationfactor * FACTOR, xrange * xtcalibrationfactor * FACTOR))
    ax0.set(ylim=(1e2, None))
    ax0.legend()

    print("Predicted Gaussian Parameters:")
    pred_ug = unflatten_biGaussian_params(pred_params)
    print("Means (Mu):", pred_ug['mu'].numpy())
    print("Covariances (Sigma):", pred_ug['Sigma'].numpy())
    print("Weights (Pi):", pred_ug['pi'].numpy())
    # cbar.set_label("Current [arb. units]")
    plt.subplots_adjust(wspace=0.4)
    # plt.tight_layout()
    # fig.show()

    # Display image_to_bigaussian_params debug info
    #biGaussianTest = image_to_bigaussian_params(images[valid_rows][ntest[idx]].reshape((2*yrange, 2*xrange)), do_current_profile, debug=True)
    #print("Bi-Gaussian Parameters for Test Image:", biGaussianTest)
    

# Create slider
interact(plot_xtcav_image_pred_current, idx=IntSlider(min=0, max=pred_test_full.shape[0]-1, step=1, value=0, layout=Layout(width='80%')));

In [ ]:
def xtcav_image_mu(idx):
    FACTOR = 1e15

    pred_params = pred_test_full.T.reshape(NCOMP,pred_test_full.shape[0])[:,idx]

    pred_ug = unflatten_biGaussian_params(pred_params)
    return (pred_ug['mu'][0][0]-pred_ug['mu'][1][0]) * xtcalibrationfactor * FACTOR  # The difference in horizontal means (mu_A_x - mu_B_x)
mu_list = []
for i in range(pred_test_full.shape[0]):
    mu_list.append(xtcav_image_mu(i))
plt.figure(figsize=(15,6))
plt.plot(mu_list)
plt.xlabel('Shot Index')
plt.ylabel('Distance between Centroids (fs)')
plt.title('Predicted Time between Two Gaussian Components in {}'.format(f"{experiment}_{runname}"))
plt.show()

In [ ]:
# Scatter plot of BC14 BLEN vs predicted peak separation
pvidx = bsaVars.index('BLEN_LI14_888_BRAW')
bc14_blen = predictor_tmp[:, pvidx]
peak_separations = mu_list
plt.figure(figsize=(8,6))
plt.scatter(bc14_blen, peak_separations, alpha=0.5)
r_squared = np.corrcoef(bc14_blen, peak_separations)[0, 1]**2
print(f"R-squared: {r_squared}")
plt.xlabel('BC14 BLEN')
plt.ylabel('Predicted Peak Separation (fs)')
plt.title(f'Predicted Peak Separation vs BC14 BLEN: {experiment}_{runname}')
plt.xlim(5000, 12500)
plt.show()

In [ ]:
from Python_Functions.functions import analyze_SYAG
syag_data = analyze_SYAG(data_struct, experiment=experiment, runname=runname, skipEOSanalysis=False, SYAGxmin=800, SYAGxmax=950, mindels=50, maxdels=500, min_prom = 1000, goosing=False, debug = False)

In [ ]:
# SYAG del vs shot index
plt.figure(figsize=(15,6))
plt.plot(syag_data['dels'])
plt.xlabel('Shot Index')
plt.ylabel('SYAG del [pixels]')
plt.title(f'SYAG del vs Shot Index: {experiment}_{runname}')
plt.show()

In [ ]:
# Scatter plot of SYAG del vs predicted peak separation
plt.figure(figsize=(8,6))
print(syag_data['dels'].size)
print(len(peak_separations))
plt.scatter(syag_data['dels'], peak_separations, alpha=0.5)
r_squared_syag = np.corrcoef(syag_data['dels'], peak_separations)[0, 1]**2
print(f"R-squared (SYAG del vs predicted peak separation): {r_squared_syag}")
plt.xlabel('SYAG Measured Peak Separation (pix)')
plt.ylabel('Predicted Peak Separation (fs)')
plt.title(f'Predicted Peak Separation vs SYAG Measured Peak Separation: {experiment}_{runname}')
plt.show()

In [ ]:
# Scatter plot of SYAG del vs predicted peak separation, filtered for blen value range
blen_filter_min = 9500
blen_filter_max = 9600
# Filter based on blen values
blen_filter = (bc14_blen >= blen_filter_min) & (bc14_blen <= blen_filter_max)
plt.figure(figsize=(8,6))
print(syag_data['dels'].size)
print(len(peak_separations))
plt.scatter(np.array(syag_data['dels'])[blen_filter], np.array(peak_separations)[blen_filter], alpha=0.5)
r_squared_syag = np.corrcoef(np.array(syag_data['dels'][blen_filter]), np.array(peak_separations)[blen_filter])[0, 1]**2
print(f"R-squared (SYAG del vs predicted peak separation): {r_squared_syag}")
plt.xlabel('SYAG Measured Peak Separation (pix)')
plt.ylabel('Predicted Peak Separation (fs)')
plt.title(f'Predicted Peak Separation vs SYAG Measured Peak Separation: {experiment}_{runname}')
plt.xlim(50, 350)
plt.show()

In [ ]:
# Scatter plot of BC14 BLEN vs predicted peak separation
step_id = 3
shots = [i for i in range(250 * step_id + 10, 250 * (step_id + 1) -10)]
bc14_blen = np.array(bc14_blen)
peak_separations = np.array(peak_separations)
plt.figure(figsize=(8,6))
plt.scatter(bc14_blen[shots], peak_separations[shots], alpha=0.5)
r_squared = np.corrcoef(bc14_blen[shots], peak_separations[shots])[0, 1]**2
print(f"R-squared: {r_squared}")
plt.xlabel('BC14 BLEN')
plt.ylabel('Predicted Peak Separation (fs)')
plt.title(f'Predicted Peak Separation vs BC14 BLEN: {experiment}_{runname}')
plt.show()

In [ ]:
# Plot BLEN BC14 vs shot index
plt.figure(figsize=(15,6))
plt.plot(bc14_blen)
plt.xlabel('Shot Index')
plt.ylabel('BC14 BLEN')
plt.title(f'BC14 BLEN vs Shot Index: {experiment}_{runname}')
plt.show()

### PCA of PV Scalars

In [ ]:
# plot bsaScalarData TCAV_LI20_2400_P and TCAV_LI20_2400_A, most important predictors.
bsaVarNames = bsaVars
var1_name = 'TCAV_LI20_2400_P'
var2_name = 'TCAV_LI20_2400_A'
var1_idx = bsaVarNames.index(var1_name)
var2_idx = bsaVarNames.index(var2_name)
print(f"Plotting BSA Scalars: {var1_name} (index {var1_idx}) and {var2_name} (index {var2_idx})")
plt.figure(figsize=(8,6))
# Very small dot and alpha
plt.scatter(predictor_tmp[:, var1_idx], predictor_tmp[:, var2_idx], alpha=0.1, s=1)
plt.xlabel(var1_name)
plt.ylabel(var2_name)
plt.title('BSA Scalar Scatter Plot')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

# Assuming bsaScalarData has shape (N_variables, N_samples) from your function
# Transpose the data so features are columns and samples are rows for scikit-learn PCA
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(predictor_tmp_cleaned)


# Determine the maximum number of components to test
# X.shape is (N_samples, N_variables). max_components = min(N_samples, N_variables) - 1 for a stable PCA
max_components = min(X.shape) // 5 

# Lists to store results
n_components_list = []
reconstruction_losses = []

# Loop through possible number of components
for k in range(1, max_components + 1):
    # 1. Initialize and fit PCA
    pca_study = PCA(n_components=k)
    pca_study.fit(X)
    
    # 2. Transform and Inverse Transform (Reconstruct)
    X_reduced = pca_study.transform(X)
    X_reconstructed = pca_study.inverse_transform(X_reduced)
    
    # 3. Calculate Reconstruction Loss (Mean Squared Error)
    loss = mean_squared_error(X, X_reconstructed)
    
    # Store results
    n_components_list.append(k)
    reconstruction_losses.append(loss)
    
# 4. Plot the Results
plt.figure(figsize=(10, 10))
plt.plot(n_components_list, reconstruction_losses, marker='o', linestyle='-')
plt.xlabel('Number of Components (k)')
plt.ylabel('Reconstruction Loss (Mean Squared Error)')
# Log scale in y-axis for better visualization
plt.yscale('log')
plt.title('PCA of BSA Scalars: Reconstruction Loss vs. Number of Components')
plt.grid(True)
# 5. Identify the "Elbow" point visually after plotting plt.show()
plt.show() 

# After plotting, the optimal number of components is the 'elbow' point.

In [ ]:

num_components = 8

pca_comp_study = PCA(n_components=num_components)
pca_comp_study.fit(X)
# 1. Get explained variance and component loadings

variance_ratios = pca_comp_study.explained_variance_ratio_
loadings = pca_comp_study.components_

# 2. Create the DataFrame for component composition
# Each row in 'loadings' is a principal component (PC)
# Each column corresponds to a feature (PV)
df_loadings = pd.DataFrame(loadings, columns=bsaVars_cleaned)

# 3. Add Component labels and Significance
component_labels = [f'PC {i+1}' for i in range(num_components)]
df_loadings.insert(0, 'Component', component_labels)
df_loadings.insert(1, 'Significance (Explained Variance Ratio)', variance_ratios)

# 4. Format the output
# The components are already ordered by significance (PC 1 is most significant)
# Format the significance column as a percentage for clarity
df_loadings['Significance (Explained Variance Ratio)'] = \
    df_loadings['Significance (Explained Variance Ratio)'].map(lambda x: f'{x:.4f} ({x*100:.2f}%)')

# Format the loadings to a fixed number of decimal places
for col in bsaVars_cleaned:
    df_loadings[col] = df_loadings[col].map(lambda x: f'{x:.4f}')

# Display the resulting table
composition_table = df_loadings.to_markdown(index=False)
print("Composition of Principal Components:")
print(composition_table)
# In the composition table, leave the most significant value in each column.
composition_table_significant = df_loadings.copy()
for col in bsaVars_cleaned:
    # Find index of two max absolute loadings for this column
    max_idx_two = df_loadings[col].astype(float).abs().sort_values(ascending=False).index[0:2].to_list()
    # Set all other values to empty string, leaving only the top two
    for idx in range(len(composition_table_significant)):
        if idx not in max_idx_two:
            composition_table_significant.at[idx, col] = ''

composition_table_significant = composition_table_significant.to_markdown(index=False)
print("Most Significant Loadings in Each Column:")
print(composition_table_significant)

In [ ]:
import seaborn as sns
# 1. Prepare the data for plotting
# The dataframe df_loadings_raw should have Components as the index and Variables as columns

df_loadings_raw = df_loadings.copy()
plot_data = df_loadings_raw
plot_data = plot_data.transpose() # Transpose to get Variables on Y-axis (often clearer)
# Debug prints
# print(plot_data.head())
# print(plot_data.dtypes)
# Remove "Component" and "Significance" rows for heatmap
plot_data = plot_data.drop(index=['Component', 'Significance (Explained Variance Ratio)'])
# Order the rows by original variable names z positions
z_positions = []
for var in plot_data.index:
    match = re.search(r'BPM_LI(\d+)_\d+_(\d+)', var)
    if match:
        z_pos = int(match.group(1))
    else:
        z_pos = float('inf')  # Place non-matching variables at the end
    z_positions.append((var, z_pos))
# Sort by z position
z_positions.sort(key=lambda x: x[1])
sorted_vars = [var for var, _ in z_positions]
plot_data = plot_data.reindex(sorted_vars)
# Convert all data to float for heatmap
plot_data = plot_data.astype(float)
# 2. Create the heatmap
plt.figure(figsize=(12, 24))
sns.heatmap(
    plot_data,
    cmap='vlag',            # 'vlag' or 'coolwarm' are good diverging colormaps (blue/red)
    center=0,               # Ensures white/transparent is at 0
    linewidths=0.5,
    linecolor='lightgray',
    annot=False,            # Set to True to display loading values in cells
    cbar_kws={'label': 'Loading Value (Correlation)'}
)

plt.title('Heatmap of Principal Component Loadings', fontsize=16)
plt.xlabel('Principal Component', fontsize=12)
plt.ylabel('Original Variable', fontsize=12)
plt.xticks(rotation=45, ha='right') # Rotate X-labels for better readability
plt.yticks(fontsize=8)
plt.tight_layout() # Adjust layout to prevent labels from being cut off
plt.show()

In [ ]:
# 1. Extract and Square the Loadings (Loadings matrix is: Components x Variables)
# loadings = pca_comp_study.components_ (from your provided code block)
squared_loadings = loadings**2

# 2. Sum Across Components (Sum columns to get total significance per variable)
# The result is an array where each element is the total squared loading for a variable
total_squared_loadings = np.sum(squared_loadings, axis=0)

# 3. Create a DataFrame for sorting
# bsaVars is the list of variable names (features)
df_var_significance = pd.DataFrame({
    'BSA Variable (PV)': bsaVars_cleaned,
    'Total Squared Loading (Significance)': total_squared_loadings
})

# 4. Sort in descending order
df_var_significance = df_var_significance.sort_values(
    by='Total Squared Loading (Significance)', 
    ascending=False
).reset_index(drop=True)

# 5. Specify the number of top variables to display
N_top = 20  # Example: display the top few most significant variables

# 6. Format and display the table
df_top_vars = df_var_significance.head(N_top)
df_top_vars['Total Squared Loading (Significance)'] = \
    df_top_vars['Total Squared Loading (Significance)'].map(lambda x: f'{x:.4f}')

inverted_table = df_top_vars.to_markdown(index=False)
print(f"Top {N_top} BSA Variables Ordered by Significance in PCA Components:")
print(inverted_table)


In [ ]:

df_filtered = df_var_significance[
    ~df_var_significance['BSA Variable (PV)'].str.contains('BPM', case=False, na=False)
]

# 5. Sort the filtered variables in descending order of significance
df_filtered = df_filtered.sort_values(
    by='Total Squared Loading (Significance)', 
    ascending=False
).reset_index(drop=True)

# 6. Select Top N and format
N_top = 20 # Display the top few most significant non-BPM variables
df_top_vars = df_filtered.head(N_top)

# Format the significance column
df_top_vars['Total Squared Loading (Significance)'] = \
    df_top_vars['Total Squared Loading (Significance)'].map(lambda x: f'{x:.4f}')

# Display the resulting table
inverted_table = df_top_vars.to_markdown(index=False)

print(f"Top {N_top} BSA Variables (EXCLUDING BPMS) Ordered by Significance in PCA Components:")
print(inverted_table)

Distance to nearest real sample

In [ ]:
from ipywidgets import FloatSlider, Layout, interact
index_to_examine = 82  # Change this index to examine different test samples
pca_vals = pca_comp_study.transform(X_test.numpy())
# Variables to be controlled (Non-BPM variables from previous ranking logic)
# ==============================================================================
# --- DYNAMIC PREDICTION FUNCTION ---
# ==============================================================================
original_predictor_reconstructed = x_scaler.inverse_transform(X_test.numpy())
def plot_dynamic_pred(**slider_values):
    idx = index_to_examine
    fig, (ax2,ax3, cx1) = plt.subplots(1,3,figsize=(12, 5), gridspec_kw={'width_ratios': [1,1, 0.02]})
    

    pred_params = pred_test_full.T.reshape(NCOMP,pred_test_full.shape[0])[:,idx]
    pred_im = biGaussian_image_from_flattened_params(pred_params).T
    if isChargePV:
        # Normalize predicted image so that its integral matches the charge
        charge_value = charge_filtered[ntest[idx]]
        pred_im_sum = np.sum(pred_im)
        pred_im = pred_im / pred_im_sum * charge_value
    im2 = ax2.imshow(pred_im*1e15, cmap = "jet", extent=(-xrange*xtcalibrationfactor*1e15, xrange*xtcalibrationfactor*1e15, 0, 2*yrange), aspect='auto')
    ax2.set(xlabel = "Time [fs]")
    ax2.set(ylabel = "y [pix]")
    ax2.set(title = "Prediction")

    # Modify the predictor for the given index based on slider values
    pca_components = []
    for var_name, slider in slider_values.items():
        # Reconstruct the original predictor values from PCA components
        pca_components.append(slider)
    pca_components = np.array(pca_components).reshape(1, -1)
    modified_predictor = pca_comp_study.inverse_transform(pca_components).flatten()


    # Scale back the modified predictor
    modified_predictor_scaled = modified_predictor.reshape(1, -1)

    # Make prediction with modified predictor
    modified_predictor_tensor = torch.tensor(modified_predictor_scaled, dtype=torch.float32)
    with torch.no_grad():
        modified_pred_scaled = model.predict(modified_predictor_tensor)
    modified_pred_full = iz_scaler.inverse_transform(modified_pred_scaled)
    modified_pred_params = modified_pred_full[0]

    # Print the modified parameters minus the original predicted parameters for comparison
    print("Difference in Parameters from Original Prediction:")
    for i, name in enumerate(["pi_1", "pi_2", "mu_1_x", "mu_1_y", "sigma_1_xx", "sigma_1_xy", "sigma_1_yy", "sigma_2_xx", "sigma_2_xy", "sigma_2_yy"]):
        diff = modified_pred_params[i] - pred_params[i]
        print(f"{name}: {diff:.4f}")

    # Print the euclidean distance to the nearest training point
    nearest_point = pca_vals[np.argmin(np.linalg.norm(pca_vals - pca_components, axis=1))]
    nearest_dist = np.linalg.norm(nearest_point - pca_components)
    print(f"Euclidean Distance to Nearest Training Point: {nearest_dist:.4f}")
    # Nearest point
    print(f"Nearest Training Point: {nearest_point}")
    modified_pred_im = biGaussian_image_from_flattened_params(modified_pred_params).T
    if isChargePV:
        # Normalize predicted image so that its integral matches the charge
        charge_value = charge_filtered[ntest[idx]]
        modified_pred_im_sum = np.sum(modified_pred_im)
        modified_pred_im = modified_pred_im / modified_pred_im_sum * charge_value
    im3 = ax3.imshow(modified_pred_im, cmap = "jet",extent=(-xrange*xtcalibrationfactor*1e15, xrange*xtcalibrationfactor*1e15, 0, 2*yrange), aspect='auto')
    ax3.set(xlabel = "Time [fs]")
    ax3.set(ylabel = "y [pix]")
    ax3.set(title = "Modified Prediction")

    cbar = fig.colorbar(im2, cax=cx1, fraction=0.16, pad=0.04)
    # cbar.set_label("Current [arb. units]")
    plt.subplots_adjust(wspace=0.4)
    # plt.tight_layout()
    # fig.show()

# ==============================================================================
# --- INTERACTIVE WIDGET SETUP ---
# ==============================================================================
# 1. Calculate min/max for sliders (using the mock raw data)

N_SAMPLES = pred_test_full.shape[0]
# 2. Setup the Widgets
widget_kwargs = {}  # Fixed index for demonstration

for i in range(pca_comp_study.components_.shape[0]):
    pv_name = f"PC{i+1}"
    pv_index = i
    min_val, max_val = pca_vals[:, pv_index].min(), pca_vals[:, pv_index].max()
    # Initial value is the mean of the range, as we cannot predict 
    # the X_test[idx] value without dynamic input in the slider setup.
    # We set it to 0.0 (or mid-range) and rely on the plot_dynamic_pred 
    # function to construct the input correctly.
    initial_value = pca_vals[index_to_examine, pv_index] # Use the value for the initial index (idx=0)
    
    widget_kwargs[pv_name] = FloatSlider(
        min=min_val, 
        max=max_val, 
        step=(max_val - min_val) / 100.0, # 100 steps granularity
        value=initial_value,
        description=pv_name,
        readout_format='.4f',
        layout=Layout(width='90%')
    )

print("--- Dynamic ML Prediction Explorer ---")
print("Adjust the sliders below to see the real-time effect on the LPS profile prediction.")

# 3. Create the Interactive Interface
# Note: The widget values are passed as keyword arguments to plot_dynamic_pred
interact(plot_dynamic_pred, **widget_kwargs)

In [ ]:
from ipywidgets import FloatSlider, Layout, interact
index_to_examine = 82  # Change this index to examine different test samples
pca_vals = pca_comp_study.transform(X_test.numpy())
# Variables to be controlled (Non-BPM variables from previous ranking logic)
# ==============================================================================
# --- DYNAMIC PREDICTION FUNCTION ---
# ==============================================================================
original_predictor_reconstructed = x_scaler.inverse_transform(X_test.numpy())
def plot_dynamic_pred(**slider_values):
    idx = index_to_examine
    # Modify the predictor for the given index based on slider values
    pca_components = []
    for var_name, slider in slider_values.items():
        # Reconstruct the original predictor values from PCA components
        pca_components.append(slider)
    pca_components = np.array(pca_components).reshape(1, -1)
    modified_predictor = pca_comp_study.inverse_transform(pca_components).flatten()
    # Plot BPMS X values, ordered by z position (4 digit integer in PV name)
    fig, (ax2,ax3) = plt.subplots(2,1,figsize=(12, 5))
    # Find BPMS names within bsaVars_cleaned
    bpm_x_indices = [i for i, var in enumerate(bsaVars_cleaned) if re.search(r'BPMS.*_X$', var)]
    # Extract z positions and sort
    bpm_z_positions = [int(re.search(r'BPMS.*_(\d+)_X$', bsaVars_cleaned[i]).group(1)) for i in bpm_x_indices]
    # Print bpm_z_positions for debugging
    print("BPM Z Positions:", bpm_z_positions)
    # print values for debugging
    print("Modified Predictor BPM X Values:", modified_predictor[bpm_x_indices])
    sorted_bpm_indices = [x for _, x in sorted(zip(bpm_z_positions, bpm_x_indices))]
    # Plot sorted_bpm_indices, at their z positions
    # These are nomalized between 0 and 1, shift down by 0.5 for visibility
    ax2.bar(bpm_z_positions, modified_predictor[sorted_bpm_indices]-0.5, color='green', label='BPM X Corresponding Values')
    ax2.set_title("Modified Predictor Values (BPM X highlighted)")
    ax2.set_ylabel("Value")
    ax2.set(ylim = (-0.5, 0.5))
    ax2.legend()

    # Plot Y values
    bpm_y_indices = [i for i, var in enumerate(bsaVars_cleaned) if re.search(r'BPMS.*_Y$', var)]
    # Extract z positions and sort
    bpm_z_positions_y = [int(re.search(r'BPMS.*_(\d+)_Y$', bsaVars_cleaned[i]).group(1)) for i in bpm_y_indices]
    sorted_bpm_indices_y = [x for _, x in sorted(zip(bpm_z_positions_y, bpm_y_indices))]
    ax3.bar(bpm_z_positions, modified_predictor[sorted_bpm_indices_y]-0.5, color='green', label='BPM Y Corresponding Values')
    ax3.set_title("Modified Predictor Values (BPM Y highlighted)")
    ax3.set_ylabel("Value")
    ax3.set(ylim = (-0.5, 0.5))
    ax3.legend()

# ==============================================================================
# --- INTERACTIVE WIDGET SETUP ---
# ==============================================================================
# 1. Calculate min/max for sliders (using the mock raw data)

N_SAMPLES = pred_test_full.shape[0]
# 2. Setup the Widgets
widget_kwargs = {}  # Fixed index for demonstration

for i in range(pca_comp_study.components_.shape[0]):
    pv_name = f"PC{i+1}"
    pv_index = i
    min_val, max_val = pca_vals[:, pv_index].min(), pca_vals[:, pv_index].max()
    # Initial value is the mean of the range, as we cannot predict 
    # the X_test[idx] value without dynamic input in the slider setup.
    # We set it to 0.0 (or mid-range) and rely on the plot_dynamic_pred 
    # function to construct the input correctly.
    initial_value = pca_vals[index_to_examine, pv_index] # Use the value for the initial index (idx=0)
    
    widget_kwargs[pv_name] = FloatSlider(
        min=min_val, 
        max=max_val, 
        step=(max_val - min_val) / 100.0, # 100 steps granularity
        value=initial_value,
        description=pv_name,
        readout_format='.4f',
        layout=Layout(width='90%')
    )

print("--- Dynamic ML Prediction Explorer ---")
print("Adjust the sliders below to see the real-time effect on the LPS profile prediction.")

# 3. Create the Interactive Interface
# Note: The widget values are passed as keyword arguments to plot_dynamic_pred
interact(plot_dynamic_pred, **widget_kwargs)